<a href="https://colab.research.google.com/github/ravindra-singh0507/Dog-and-Cat-implementation/blob/main/Dog-and-Cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.17.1'

Part 1- Data PreProcessing

In [ ]:
import zipfile

with zipfile.ZipFile(r'kaggle cats and dogs_5340.zip', 'r') as zip_ref:
    zip_ref.extractall(r'path_to_destination')


In [ ]:
train_path = r"path_to_destination\PetImages"


In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
train_path = r"path_to_destination/PetImages"

training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 25000 images belonging to 2 classes.


In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_set= test_datagen.flow_from_directory('path_to_destination/PetImages',target_size=(64,64),batch_size=32,class_mode='binary')

Found 25000 images belonging to 2 classes.


Part 2: Building the CNN

Initialising the CNN

In [ ]:
cnn=tf.keras.models.Sequential()

step 1: convoltion

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


step 2: pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

step 4: full connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Output layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Part 3- Training the CNN

compliling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
import os
from PIL import UnidentifiedImageError

def load_image_safe(image_path):
  """Loads an image and handles potential errors."""
  try:
    img = tf.keras.utils.load_img(image_path, target_size=(64, 64))
    return img
  except UnidentifiedImageError as e:
    print(f"Error loading image: {image_path}")
    print(e)
    return None

def generate_data(directory, target_size=(64, 64), batch_size=32, class_mode='binary'):
  """Generates image data with error handling."""
  image_paths = []
  labels = []

  for class_name in os.listdir(directory):
    class_path = os.path.join(directory, class_name)
    if os.path.isdir(class_path):
      for filename in os.listdir(class_path):
        image_path = os.path.join(class_path, filename)
        img = load_image_safe(image_path)
        if img is not None:
          image_paths.append(image_path)
          labels.append(class_name)

  datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
  return datagen.flow_from_dataframe(
      pd.DataFrame({'filename': image_paths, 'class': labels}),
      x_col='filename',
      y_col='class',
      target_size=target_size,
      batch_size=batch_size,
      class_mode=class_mode
  )
!pip install pandas
import pandas as pd
training_set = generate_data(train_path)
test_set = generate_data('path_to_destination/PetImages')

Error loading image: path_to_destination/PetImages/Dog/Thumbs.db
cannot identify image file <_io.BytesIO object at 0x7f885a78c5e0>
Error loading image: path_to_destination/PetImages/Dog/11702.jpg
cannot identify image file <_io.BytesIO object at 0x7f88617f5580>


/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Error loading image: path_to_destination/PetImages/Cat/Thumbs.db
cannot identify image file <_io.BytesIO object at 0x7f88617f5580>
Error loading image: path_to_destination/PetImages/Cat/666.jpg
cannot identify image file <_io.BytesIO object at 0x7f88611bc5e0>
Found 24998 validated image filenames belonging to 2 classes.
Error loading image: path_to_destination/PetImages/Dog/Thumbs.db
cannot identify image file <_io.BytesIO object at 0x7f885a78c5e0>
Error loading image: path_to_destination/PetImages/Dog/11702.jpg
cannot identify image file <_io.BytesIO object at 0x7f8863ed5fd0>


/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Error loading image: path_to_destination/PetImages/Cat/Thumbs.db
cannot identify image file <_io.BytesIO object at 0x7f885a795580>
Error loading image: path_to_destination/PetImages/Cat/666.jpg
cannot identify image file <_io.BytesIO object at 0x7f88617f5580>
Found 24998 validated image filenames belonging to 2 classes.


In [41]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


273/782 ━━━━━━━━━━━━━━━━━━━━ 1:50 216ms/step - accuracy: 0.6555 - loss: 0.6193

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


782/782 ━━━━━━━━━━━━━━━━━━━━ 266s 339ms/step - accuracy: 0.6735 - loss: 0.6017 - val_accuracy: 0.7439 - val_loss: 0.5193
Epoch 2/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 268s 342ms/step - accuracy: 0.7472 - loss: 0.5147 - val_accuracy: 0.7724 - val_loss: 0.4796
Epoch 3/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 266s 340ms/step - accuracy: 0.7808 - loss: 0.4635 - val_accuracy: 0.7604 - val_loss: 0.4923
Epoch 4/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 316s 332ms/step - accuracy: 0.7937 - loss: 0.4445 - val_accuracy: 0.8047 - val_loss: 0.4196
Epoch 5/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 259s 331ms/step - accuracy: 0.8062 - loss: 0.4236 - val_accuracy: 0.8106 - val_loss: 0.4141
Epoch 6/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 262s 334ms/step - accuracy: 0.8163 - loss: 0.4045 - val_accuracy: 0.8251 - val_loss: 0.3840
Epoch 7/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 269s 344ms/step - accuracy: 0.8228 - loss: 0.3898 - val_accuracy: 0.8383 - val_loss: 0.3651
Epoch 8/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 310s 396ms/step - accuracy: 0.8292 - loss: 0.37

Part 4- Making single Prediction

In [49]:
import numpy as np
from keras.preprocessing import image
test_image = tf.keras.utils.load_img(
    r"/content/path_to_destination/PetImages/Cat/10001.jpg",
    target_size=(64, 64)
)
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
print(training_set.class_indices)
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
{'Cat': 0, 'Dog': 1}


In [50]:
print(prediction)

cat
